In [1]:
import cv2
import numpy as np
import os
from scipy import stats

In [2]:
def distance(x,y):
    return(np.sqrt(sum((x-y)**2)))

In [3]:
def KNN(train,test,k=5):
    dist=[]
    #print(train.shape)
    #print(test.shape)
    for i in range(train.shape[0]):
        ix=train[i,:-1]
        iy=train[i,-1]
        #print(ix.shape)
        
        d=distance(test,ix)
        dist.append((d,iy))
        
    dist_list=sorted(dist)
    #print(dist_list)
    dist_list=dist_list[:k]
    #print(dist_list)
    dist_list=np.array(dist_list)
    #print(dist_list)
        
    vals=dist_list[:,1]
    pred=stats.mode(vals)
    return int(pred[0])
        

In [7]:
cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier("C:/Users/surender/Face Detect/haarcascade_frontalface_alt.xml")
path='./face_data/'

face_data=[]
labels=[]

class_id=0
names={}

for fx in os.listdir(path):
    if fx.endswith('.npy'):
        names[class_id] = fx[:-4]
        print("Loaded "+fx)
        data_item = np.load(path+fx)
        face_data.append(data_item)
        
        target = class_id*np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)

face_dataset = np.concatenate(face_data,axis=0)
face_labels = np.concatenate(labels,axis=0).reshape((-1,1))
trainset = np.concatenate((face_dataset,face_labels),axis=1)
print(trainset)



while True:
    ret,frame=cap.read()
    
    if(ret==False):
        continue

    #frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(frame,1.3,5)
    
    if(len(faces)==0):
        continue
    
    for face in faces:
        x,y,w,h=face
        
        offset=10
        face_section=frame[y-offset:y+offset+h,x-offset:x+offset+w]
        face_section=cv2.resize(face_section,(100,100))
        face_section=np.array(face_section).reshape((-1,))
        
        out = KNN(trainset,face_section.flatten())
        pred_name = names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)
    
    cv2.imshow("frame",frame)
    
    key_press=cv2.waitKey(1) & 0XFF
    if(key_press==ord('q')):
        break

cap.release()
cv2.destroyAllWindows()

Loaded Lakshya.npy
Loaded Shahrukh.npy
Loaded Shrey.npy
[[230. 234. 234. ...  25.  25.   0.]
 [ 26.  24.  28. ...  33.  30.   0.]
 [ 26.  24.  28. ...  33.  30.   0.]
 ...
 [165. 165. 172. ...  56.  67.   2.]
 [165. 165. 172. ...  56.  67.   2.]
 [146. 141. 158. ...  82.  89.   2.]]
